In [18]:
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re, unicodedata
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', -1)

In [118]:
def transform(line):
    contraction_mapping = {"ive":"i have","dont":"do not","cant":"can not","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "havent":"have not","haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
    word_list = []
    """Convert all characters to lowercase"""
    
    line = re.sub(r'\?+', "? ", line)
    line = re.sub(r'\!+', "! ", line)
    line = re.sub(r'\.+\n*', ". ", line)
    line = re.sub(r',[\w+\s]', ", ", line)
    line = line.lower().replace('"',' " ').replace(':',' : ').replace('\n', ' ').replace('/', ' ').replace('_', ' ')
    if "." not in line[-3:] and "?" not in line[-3:] and "!" not in line[-3:]:
        line = line + '.'
    for word in line.split():
        new_word = word
        """Remove punctuation except for "?" and "!" from list of tokenized words"""
        if contraction_mapping.__contains__(new_word):
            new_word = contraction_mapping[new_word]
        word_list.append(new_word)
    return ' '.join(word_list)

def normalize(word):
    """Remove non-ASCII characters from list of tokenized words"""
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    new_word = re.sub(r'[^\w\s\.\!\?\:\,\-\(\)}]', '', new_word)
    lemmatizer = WordNetLemmatizer()
    new_word = lemmatizer.lemmatize(new_word, pos='v')
    return new_word

In [119]:
def sent_preprocessing(sent):
    return " ".join([normalize(word) for word in word_tokenize(transform(sent)) if normalize(word) != ""]).strip()

In [21]:
data = []
with open("../Crawling/PostProcessing/Abdominal_Disorders_merged.jl") as f:
#     for line in f.readlines():
       df = pd.DataFrame(json.loads(line) for line in f)